<a href="https://colab.research.google.com/github/MaLuisa7/Applied-Deep-Learning-classes/blob/main/Copia_de_RF_lag0_lag1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from matplotlib.ticker import MaxNLocator
from functools import reduce

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.gridspec import GridSpec
from sklearn.tree import export_graphviz
import graphviz

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression
from sklearn.metrics import make_scorer, mean_absolute_error ,mean_squared_error,r2_score



In [ ]:
#------------------------------------------------------------------FUNCTIONS
def select_oneComb_Nniv(df_consolidado, nivN, names_comb):
    query_oneComb = ''
    for n in range(0, len(nivN)):
        subquery = '(' + str(nivN[n]) + '== "' + str(names_comb[n]) + '") & '
        if n == (len(nivN) - 1):
            subquery = '(' + str(nivN[n]) + '== "' + str(names_comb[n]) + '")   '
        query_oneComb += subquery

    oneComb_nivN = df_consolidado.query(query_oneComb)
    return oneComb_nivN

def outliers(combstory, name_x):
    '''
    :param combstory:es la seleccion de la historia de una combinacion
    :param name_x: nombre de la variable de tipo continua que se quiere omitir
    :return: devuelve el consolidado de la combinacion unica sin los datos atipicos
    '''
    for xname in name_x:
        x = combstory.loc[:, xname]
        print(combstory.shape)
        data_mean_x, data_std_x = np.mean(x), np.std(x)
        cut_off_x = data_std_x * 2.5
        lower_x, upper_x = data_mean_x - cut_off_x, data_mean_x + cut_off_x
        query1 = "(" + xname + "> @lower_x) & (" + xname + " < @upper_x)"
        combstory = combstory.query(query1)
    return combstory



# Calculate Mean Absolute Percentage Error (MAPE)
def calculate_mape(y_true, y_pred):
    actual, forecast = np.array(y_true), np.array(y_pred)
    mask = actual != 0
    return np.mean(np.abs((y_true - y_pred) / y_true)[mask]) * 100

def lags_datanorm(df_final):
    data = df_final.loc[:, lst_levels + lst_temp + lst_varsnorm].copy()
    lst_combs = []

    for n in range(len(unique_comb)):
        comb_name = unique_comb.iloc[n, :].to_list()
        comb_story = select_oneComb_Nniv(data, lst_levels, comb_name).copy()

        for i in range(3):  # replace '3' with your specific range value if needed
            comb_story.loc[:, 'sellin_norm_lag_' + str(i)] = comb_story['sellin_norm'].shift(i)

        lst_combs.append(comb_story)

    data_final = pd.concat(lst_combs)
    return data_final

def makeplot(rf, lst_feat):
    dot_data = export_graphviz(rf.estimators_[0], out_file=None,
                               feature_names=lst_feat,
                               filled=True, rounded=True,
                               special_characters=True)

    # Create a Graphviz source object
    graph = pydotplus.graph_from_dot_data(dot_data)

    # Create and display an image
    image_path = 'random_forest_tree.png'
    graph.write_png(image_path)
    display(Image(filename=image_path))



In [ ]:
path_exp = "/content/drive/MyDrive/tesinita/data_initial_prep/"
name_train = 'data_train.csv'
name_test = 'data_test.csv'
name_whole = 'data_complete.csv'
df_final = pd.read_csv(path_exp + name_whole)
df_test = pd.read_csv(path_exp + name_train)
df_train = pd.read_csv(path_exp + name_test)

In [ ]:

#------------------------------------------------------------------LISTS ANDS VARIABLES
lst_levels = ['n1', 'n2', 'n3', 'n4']
lst_temp = ['year', 'month', 'week',  'year_month', 'year_week' ]
lst_vars = [ 'sellin', 'sellout']
lst_varsnorm = ['sellin_norm', 'sellout_norm']
date_train = '2023 W16'
name_y =  'sellout_norm'

lst_feat =[ 'sellin_norm_lag_0',  'sellin_norm_lag_1' ]


In [ ]:
unique_comb = df_final.loc[:, lst_levels].drop_duplicates().reset_index(drop=True)
print(unique_comb.shape)
unique_comb.head()

(155, 4)


,n1,n2,n3,n4
0,f1,c1,s1,ss_nr
1,f1,c2,s2,ms_nr
2,f1,c2,s2,ss_nr
3,f1,c2,s3,ms_nr
4,f1,c2,s3,ss_nr


In [ ]:
#----------------------------------------------------------------DATA PREP
data_final =  lags_datanorm (df_final)

df_train = data_final.query("year_week <= @date_train")
df_test = data_final.query("year_week > @date_train")
df_train = df_train.loc[:, lst_levels + lst_temp + lst_varsnorm + lst_feat]
df_test = df_test.loc[:, lst_levels + lst_temp + lst_varsnorm + lst_feat]

In [ ]:
#-------------SETTINGS
# Define an extended parameter grid
param_grid = {
    'n_estimators': [20, 40, 60, 80, 100, 200, 300], # número de árboles a utilizar
    'max_depth': [None, 1, 2, 3, 4, 5], # máximo de profundidad
    'min_samples_split': [5, 10], # mínimo de muestras en cada separación
}
lst_bestparams = []
lst_featimp = []
lst_errors = []
lst_results = []  # lst_results = lst_levels + lst_bestparams + lst_featimp + lst_errors
displaymakeplot = False

#-------------ALGORITHM
for n in range( 0 , len(unique_comb)):
  combname = unique_comb.iloc[n, :].to_list()
  print(f'combname: {combname}')
  combstory_train = select_oneComb_Nniv(df_train, lst_levels, combname).dropna()
  #quitar outliers
  combstory_train = outliers(combstory_train, lst_feat )

  combstory_test = select_oneComb_Nniv(df_test, lst_levels, combname).dropna()

  X_train = combstory_train.loc[:, lst_feat]
  y_train = combstory_train.loc[:, name_y]
  X_test = combstory_test.loc[:, lst_feat]
  y_test = combstory_test.loc[:, name_y]

  # GRIDSEARCH
  grid_search = GridSearchCV(estimator=RandomForestRegressor(),
                            param_grid=param_grid,
                            cv=5,
                            n_jobs=-1,
                            scoring='neg_mean_absolute_error')
  # Fit the GRIDSEARCH
  grid_search.fit(X_train, y_train)

  # Get the best parameters
  n_estimators = grid_search.best_params_['n_estimators']
  min_samples_split = grid_search.best_params_['min_samples_split']
  max_depth = grid_search.best_params_['max_depth']
  params = [n_estimators, min_samples_split, max_depth]
  lst_bestparams.extend(params)

  # Fit a RF with the best parameters
  rf = RandomForestRegressor(n_estimators=n_estimators,
                            min_samples_split=min_samples_split,
                            max_depth=max_depth,
                            random_state=7)
  rf.fit(X_train, y_train)

  # Get the feature importances
  feature_importances = list(rf.feature_importances_)
  lst_featimp.extend(feature_importances)

  # Make the figure if you want
  if displaymakeplot:
      makeplot(rf, lst_feat)

  # Get the errors metrics
  y_pred = rf.predict(X_test)
  combstory_test['y_pred'] = y_pred

  # Get the residuals
  y_true = combstory_test.loc[:, name_y]
  y_pred = combstory_test.loc[:, 'y_pred']
  r2 = r2_score(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  mae = mean_absolute_error(y_true, y_pred)
  mape = calculate_mape(y_true, y_pred)
  residuals_mean = np.mean(y_true - y_pred)
  error_metrics = [r2, mse, mae, mape, residuals_mean]
  lst_errors.extend(error_metrics)

  # Flatten lst_results into a single list
  lst_results.append(lst_levels + lst_bestparams + lst_featimp + lst_errors)


combname: ['f1', 'c1', 's1', 'ss_nr']
combname: ['f1', 'c2', 's2', 'ms_nr']
combname: ['f1', 'c2', 's2', 'ss_nr']
combname: ['f1', 'c2', 's3', 'ms_nr']
combname: ['f1', 'c2', 's3', 'ss_nr']
combname: ['f1', 'c3', 's4', 'ss_nr']
combname: ['f1', 'c3', 's5', 'ms_nr']
combname: ['f1', 'c3', 's5', 'ss_nr']
combname: ['f1', 'c4', 's6', 'ms_r']
combname: ['f1', 'c4', 's7', 'ms_nr']
combname: ['f1', 'c4', 's7', 'ss_nr']
combname: ['f1', 'c4', 's7', 'ss_r']
combname: ['f1', 'c5', 's8', 'ms_nr']
combname: ['f1', 'c5', 's8', 'ms_r']
combname: ['f1', 'c5', 's8', 'ss_nr']
combname: ['f1', 'c5', 's8', 'ss_r']
combname: ['f1', 'c5', 's9', 'ms_nr']
combname: ['f1', 'c5', 's9', 'ms_r']
combname: ['f1', 'c5', 's9', 'ss_nr']
combname: ['f1', 'c5', 's9', 'ss_r']
combname: ['f2', 'c1', 's1', 'ss_nr']
combname: ['f2', 'c2', 's2', 'ms_nr']
combname: ['f2', 'c2', 's2', 'ss_nr']
combname: ['f2', 'c2', 's3', 'ms_nr']
combname: ['f2', 'c2', 's3', 'ss_nr']
combname: ['f2', 'c3', 's4', 'ss_nr']
combname: ['f2', '

In [ ]:
df_results_rf_3x =pd.DataFrame(data = lst_results,  columns =  lst_levels + ['n_estimators', 'min_samples_split', 'max_depth'] + lst_feat +
                             ['r2', 'mse', 'mae', 'mape', 'residuals_mean'] )

[0.05341653686156083,
 0.17086904208228967,
 29.807908081576763,
 -0.1439712285602271]

In [ ]:
#Este cpdigo tarda mas de 1hora 31 min